In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('dataset-0510/train.csv')
test = pd.read_csv('dataset-0510/test.csv')

In [ ]:
col = ['divorce_rate', 'parking_way', 'city', 'highschool_rate', 'elementary_rate', 'junior_rate', 'town', 'town_area']

In [ ]:
train_specific_col = train[col]

In [ ]:
#missing value counts in each of these columns
Isnull = train_specific_col.isnull().sum()/len(train)*100
Isnull = Isnull[Isnull>0]
Isnull.sort_values(inplace=True, ascending=False)
Isnull

In [ ]:
y = train['total_price']

In [ ]:
X = train_specific_col.values
y = y.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)*100

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1000)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)*100

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor(n_estimators=100, max_depth=4)

In [ ]:
GBR.fit(X_train, y_train)

In [ ]:
GBR.score(X_test, y_test)*100